In [1]:
from IPython.core.display import display, HTML
display(HTML("""<style> .container {width:96% !important;}</style>"""))

from IPython.display import IFrame

In [2]:
import pandas as pd
import numpy as np
from __future__ import division

In [3]:
import xgboost as xgb

In [4]:
import sys
sys.path.insert(0,'../')
from utils.paths import *
# mod = reload(sys.modules['utils.paths'])
# vars().update(mod.__dict__) 

In [5]:
import preprocessing as pp
reload(pp);
import graphs as gg
reload(gg);

In [16]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['e']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# Experiment on model life time

In [6]:
nat = pd.read_csv(path_SBA + 'SBAnational_new.csv', sep = ';', low_memory=False)

In [15]:
print nat.shape
# Extra features
nat['Expanding'] = nat.CreateJob.apply(pp.expanding)
nat['Retaining'] = nat.CreateJob.apply(pp.retaining)
nat['Expanding_ratio'] = nat.apply(lambda x: pp.expanding_ratio(x['CreateJob'], x['NoEmp']), axis= 1)
nat['Retaining_ratio'] = nat.apply(lambda x: pp.retaining_ratio(x['RetainedJob'], x['NoEmp']), axis= 1)

use_col = ['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Bank', 'BankState', 
           'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist', 
           'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr', 
           'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross', 
           'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv', 
           'default', 'Zip5d', 'Zip3d', 'SBA_ratio', 'RealEstate', 
           'NAICS_default_rate', 'NAICS_group', 'suffix', 'Loan_age', 'Previous_loan', 'default_times',
           'fips', 'BusinessType',
           'Expanding', 'Retaining', 'Expanding_ratio', 'Retaining_ratio'
          ]

nat = nat[use_col]
print nat.shape


(897137, 46)
(897137, 42)


In [43]:
from sklearn import model_selection
from sklearn.metrics import roc_curve, roc_auc_score

def tuning_grades(num_grades, prob):
    Percentile = list(np.linspace(0, 100, num_grades+1))
    thresholds = [np.percentile(prob, i) for i in Percentile]
    thresholds[0] = 0
    thresholds[-1] = 1
    thresholds = [round(i, 3) for i in thresholds]
    return thresholds

class ml_model(object):
    def __init__(self, portfolio_period, operation_period1, operation_period2):
        portfolio = nat[nat.ApprovalFY.isin(portfolio_period)].reset_index(drop = True)
        operation1 = nat[nat.ApprovalFY.isin(operation_period1)].reset_index(drop = True)
        operation2 = nat[nat.ApprovalFY.isin(operation_period2)].reset_index(drop = True)
        
        self.model_performance = {'portfolio_period' : portfolio_period,
                                  'operation_period1' : operation_period1,
                                  'operation_period2' : operation_period2
                                 }
        self.model_output = {'portfolio_period' : portfolio_period,
                             'operation_period1' : operation_period1,
                             'operation_period2' : operation_period2,
                            }
        
        Train, Test = model_selection.train_test_split(portfolio, 
                                                       test_size = 0.25, 
                                                       random_state = 1868,
                                                       stratify = portfolio.default 
                                                       )
        # Preprocessing train set
        features = Train
        target = Train.default
        drop = ['LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'ApprovalFY', 'ChgOffDate', 'DisbursementDate',
                'DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv', 'SBA_ratio',
                'default', 'FranchiseCode', 'Term', 'NAICS']
        categorical = ['City', 'State', 'Zip5d', 'Zip3d', 'Bank', 'BankState', 'RevLineCr', 
                       'LowDoc', 'NAICS_group', 'suffix', 'fips', 'BusinessType',
                       'Expanding_ratio', 'Retaining_ratio'
                      ]
        dict_categorical, features = pp.extract_train_features(features,
                                                            drop,
                                                            categorical)
        
        X_train, X_test, y_train, y_test = model_selection.train_test_split(features, 
                                                                    target, 
                                                                    test_size = 0.25, 
                                                                    random_state=3776,
                                                                    stratify=target 
                                                                   )
        dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        dtest = xgb.DMatrix(X_test.values, y_test.values)
        num_rounds = 1100
        # num_rounds = 2000
        params = {'silent':1, 
                  'eta':0.01, 
                  'max_depth':11, 
                  'subsample': 0.6, 
                  'colsample_bytree': 0.4,
                  'min_child_weight':1, 
                  'objective':'binary:logistic', 
                  'eval_metric':'auc', 
                  'seed':2017, 
                  'gamma':0.1,
                  'nthread':-1}
        watchlist = [(dtrain, 'train'),(dtest,'validation')]
        bst=xgb.train(params, dtrain, num_rounds, watchlist, early_stopping_rounds = 50, verbose_eval = False);
        num_rounds = bst.best_iteration
#         print bst.best_iteration, bst.best_score
        self.model_performance['best_iteration'] = bst.best_iteration
        self.model_performance['best_score'] = bst.best_score
        
        # Use all the train data to train the model
        X_train_matrix = features.values
        #SKLEARN
        clf_xgb = xgb.XGBClassifier(silent = params['silent'],
                                    learning_rate = params['eta'],  
                                    max_depth = params['max_depth'], 
                                    subsample = params['subsample'], 
                                    colsample_bytree = params['colsample_bytree'],
                                    min_child_weight = params['min_child_weight'], 
                                    objective = params['objective'], 
                                    n_estimators = num_rounds,
                                    seed = params['seed'],
                                    nthread = params['nthread'],
                                    gamma = params['gamma']
                                                    )
        clf_xgb.fit(X_train_matrix, 
                    target, 
                    eval_metric ='auc')
        # Validate model with test set
        # Preprocessing test set
        test_X = Test.copy()
        # Preprocessing
        test_bas = pp.extract_test_features(test_X,
                                 drop,
                                 categorical,
                                 self.dict_categorical)
        # Prediction
        for col in features.columns:
            if col not in test_bas.columns:
                print 'MISSING COLUMN: ',col
        test_bas= test_bas[features.columns]
        X_test_matrix = test_bas.values
#         print X_train_matrix.shape, X_test_matrix.shape
        y_test_pred_xgb = clf_xgb.predict_proba(X_test_matrix)
        result_table_test = test_X[['LoanNr_ChkDgt', 'Name', 'ApprovalFY', 'State', 'default', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv', 'SBA_ratio']]
        result_table_test.loc[:, 'prob'] = y_test_pred_xgb[:,1]
        fpr, tpr, _ = roc_curve(result_table_test.default, result_table_test.prob)
        auc_plot = roc_auc_score(result_table_test.default, result_table_test.prob)   
        self.model_output['test_fpr'] = fpr
        self.model_output['test_tpr'] = tpr
        self.model_performance['test_auc_plot'] = auc_plot
        
        # Assign grades using test set
        prob_th = tuning_grades(5, result_table_test.prob)
        grades = [str(g) for g in range(1,6)]
        result_table_test.loc[:, 'Grade'] = pd.cut(result_table_test.prob, bins=prob_th, labels=grades)
        result_table_test.loc[:, 'Grade'] = result_table_test['Grade'].astype('int')
        self.model_output['result_table_test'] = result_table_test
        
        # Preprocessing operation1 set
        proj_X = operation1.copy()        
        proj_bas = pp.extract_test_features(proj_X,
                                    drop,
                                    categorical,
                                    dict_categorical)
        
        for col in features.columns:
            if col not in proj_bas.columns:
                print 'MISSING COLUMN: ',col
        
        proj_bas= proj_bas[features.columns]
        X_proj_matrix = proj_bas.values
#         print X_train_matrix.shape, X_test_matrix.shape, X_proj_matrix.shape

        y_proj_pred_xgb = clf_xgb.predict_proba(X_proj_matrix)
        temp = pd.DataFrame(y_proj_pred_xgb)
        
        result_table_proj = proj_X[['LoanNr_ChkDgt', 'Name', 'ApprovalFY', 'State', 'default', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv', 'SBA_ratio']]
        result_table_proj.loc[:, 'prob'] = y_proj_pred_xgb[:,1]
        
        fpr, tpr, _ = roc_curve(result_table_proj.default, result_table_proj.prob)
        auc_plot = roc_auc_score(result_table_proj.default, result_table_proj.prob)   
        self.model_output['oper1_fpr'] = fpr
        self.model_output['oper1_tpr'] = tpr
        self.model_performance['oper1_auc_plot'] = auc_plot
        
        result_table_proj.loc[:, 'Grade'] = pd.cut(result_table_proj.prob, bins=prob_th, labels=grades)
        result_table_proj.loc[:, 'Grade'] = result_table_proj['Grade'].astype('int')
        self.model_output['result_table_oper1'] = result_table_proj
        
        # Preprocessing operation2 set
        proj_X = operation2.copy()        
        proj_bas = pp.extract_test_features(proj_X,
                                    drop,
                                    categorical,
                                    dict_categorical)
        
        for col in features.columns:
            if col not in proj_bas.columns:
                print 'MISSING COLUMN: ',col
        
        proj_bas= proj_bas[features.columns]
        X_proj_matrix = proj_bas.values
#         print X_train_matrix.shape, X_test_matrix.shape, X_proj_matrix.shape

        y_proj_pred_xgb = clf_xgb.predict_proba(X_proj_matrix)
        temp = pd.DataFrame(y_proj_pred_xgb)
        
        result_table_proj = proj_X[['LoanNr_ChkDgt', 'Name', 'ApprovalFY', 'State', 'default', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv', 'SBA_ratio']]
        result_table_proj.loc[:, 'prob'] = y_proj_pred_xgb[:,1]
        
        fpr, tpr, _ = roc_curve(result_table_proj.default, result_table_proj.prob)
        auc_plot = roc_auc_score(result_table_proj.default, result_table_proj.prob)   
        self.model_output['oper2_fpr'] = fpr
        self.model_output['oper2_tpr'] = tpr
        self.model_performance['oper2_auc_plot'] = auc_plot
        
        result_table_proj.loc[:, 'Grade'] = pd.cut(result_table_proj.prob, bins=prob_th, labels=grades)
        result_table_proj.loc[:, 'Grade'] = result_table_proj['Grade'].astype('int')
        self.model_output['result_table_oper2'] = result_table_proj
        
        

In [44]:
experiments = [([2003, 2004], [2005], [2006]),
               ([2004, 2005], [2006], [2007]),
               ([2005, 2006], [2007], [2008]),
               ([2006, 2007], [2008], [2009]),
               ([2007, 2008], [2009], [2010]),
               ([2008, 2009], [2010], [2011]),
               ([2009, 2010], [2011], [2012]),
               ([2010, 2011], [2012], [2013])
              ]

In [45]:
runs = []
for e in experiments[:1]:
    p_period = e[0]
    o_period1 = e[1]
    o_period2 = e[2]
    print p_period, o_period1, o_period2
    run = ml_model(p_period, o_period1, o_period2)
    print run.model_performance
    runs.append(run)
    

[2003, 2004] [2005] [2006]
-----> Extract train features <------
dropping unwanted columns
transforming categorical variables
done


AttributeError: 'ml_model' object has no attribute 'dict_categorical'